## Mirandata - evasão em cursos de engenharia

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

par_datas = ((2009,2013), (2010, 2014), (2011,2015), (2012, 2016))


def evasao_ponderada(df_filtrado, par_datas):
    print(df_filtrado.NO_CURSO.value_counts())
    df = df_filtrado
    ev = calcula_evasao(df, par_datas)
    d = pd.DataFrame(ev)
    d_pos = d[d[2]>0] # Excluí os casos em que o numero de vagas diminuiu. São ~ 0.5% das IEs
    # Evasao = peso da IE * taxa de evasao
    d_pos_evasao = d_pos[2]*d_pos[3]
    d_pos_evasao= d_pos_evasao/np.sum(d_pos[3])
    return d_pos_evasao
    
    
def calcula_evasao(df_filtrado, par_datas):
    df = df_filtrado
    gb = df.groupby(["CO_IES",  "CO_CURSO", "ano_censo"]).sum()
    gb_mini = df.groupby(["CO_IES",  "CO_CURSO"]).sum()
    evasao = []
    for par in gb_mini.index:
        for datas in par_datas:
            evasao_par = calcula_evasao_gb(gb, par[0], par[1], datas[0], datas[1])
            if evasao_par != None:
                evasao.append(evasao_par)   
    return evasao
    
       
def calcula_evasao_gb(gb, ies, curso, ano_inicial, ano_final):
    #print(gb.loc[(ies, curso),:].index ) 
    if (ano_inicial in gb.loc[(ies, curso),:].index ) and (ano_final in gb.loc[(ies, curso),:].index):
        if pd.notnull(gb.loc[(ies,curso,ano_final),:].QT_CONCLUINTE_CURSO) and pd.notnull(gb.loc[(ies,curso,ano_inicial),:].QT_INGRESSO_CURSO):
            saida = gb.loc[(ies,curso,ano_final),:].QT_CONCLUINTE_CURSO
            entrada = gb.loc[(ies,curso,ano_inicial),:].QT_INGRESSO_CURSO
            evasao = (entrada - saida)/entrada
            evasao_peso = entrada
            #print("saida: {0}, ingresso{1} ".format(saida, entrada))
            if evasao_peso!=0:
                #Não teve ingresso 5 anos atrás
                return ies, curso, evasao, evasao_peso
            else:
                return None
        else:
            #print("Não foi possível calcular evasão")
            pass

    return None
    



In [3]:
dados = pd.read_excel("engenharias_2009_a_2016.xlsx")

In [4]:
comps = dados.loc[dados.NO_CURSO.str.contains("COMPUTAÇÃO"),:]
mec = dados.loc[dados.NO_CURSO.str.contains("MECÂNICA"),:]
mex = dados.loc[dados.NO_CURSO.str.contains("TRÔNICA")|dados.NO_CURSO.str.contains("CONTROLE") ,:]

In [5]:
comps.NO_CURSO.value_counts()

ENGENHARIA DE COMPUTAÇÃO                                   862
ENGENHARIA DA COMPUTAÇÃO                                   261
ENGENHARIA DE COMPUTAÇÃO E INFORMAÇÃO                        7
ENGENHARIA DE COMPUTAÇÃO - ÊNFASE SISTEMAS CORPORATIVOS      3
ENGENHARIA DE COMPUTAÇÃO - ENGENHARIA DA COMPUTAÇÃO          3
ENGENHARIA DA COMPUTAÇÃO - COMPUTAÇÃO APLICADA               2
ENGENHARIA DA COMPUTAÇÃO - AUTOMAÇÃO INDUSTRIAL              2
ENGENHARIA DA COMPUTAÇÃO - SISTEMAS E COMPUTAÇÃO             2
ENGENHARIA DA COMPUTAÇÃO - TELEMÁTICA                        1
Name: NO_CURSO, dtype: int64

In [6]:
mec.NO_CURSO.value_counts()

ENGENHARIA MECÂNICA                                    2080
ENGENHARIA MECÂNICA AERONÁUTICA                          13
ENGENHARIA MECÂNICA - ÊNFASE EM MATERIAIS METÁLICOS       7
ENGENHARIA MECÂNICA EMPRESARIAL                           7
ENGENHARIA MECÂNICA - ÊNFASE EM MECÂNICA PLENA            7
ENGENHARIA MECÂNICA E DE AUTOMÓVEIS                       7
ENGENHARIA MECÂNICA - ÊNFASE EM PROJETOS                  6
ENGENHARIA MECÂNICA - ENGENHARIA MECÂNICA                 6
ENGENHARIA MECÂNICA E DE ARMAMENTO                        5
ENGENHARIA MECÂNICA - ÊNFASE EM AERONAVES                 4
ABI - ENGENHARIA MECÂNICA                                 3
ENGENHARIA MECÂNICA DE ARMAMENTO                          2
ENGENHARIA INDUSTRIAL MECÂNICA                            2
ENGENHARIA MECÂNICA - MECÂNICA E AUTOMOBILÍSTICA          2
ENGENHARIA MECÂNICA - CONTROLE E AUTOMAÇÃO                2
ENGENHARIA MECÂNICA - ÊNFASE EM MECATRÔNICA               2
ENGENHARIA - ENGENHARIA MECÂNICA        

In [7]:
mex.NO_CURSO.value_counts()

ENGENHARIA DE CONTROLE E AUTOMAÇÃO                                         966
ENGENHARIA MECATRÔNICA                                                     187
ENGENHARIA DE CONTROLE E AUTOMACAO                                          41
ENGENHARIA DE CONTROLE E AUTOMAÇÃO - MECATRÔNICA                            38
ENGENHARIA - ENGENHARIA DE CONTROLE E AUTOMACAO                             11
ENGENHARIA DE AUTOMAÇÃO E CONTROLE                                          11
ENGENHARIA DE CONTROLE E AUTOMACAO  MECATRONICA                             10
ENGENHARIA DE CONTROLE E AUTOMACAO MECATRONICA                               8
ENGENHARIA DE MECATRÔNICA                                                    7
ENGENHARIA DE CONTROLE E AUTOMAÇÃO DE PROCESSOS                              7
MECATRÔNICA                                                                  3
ENGENHARIA - ENGENHARIA DE AUTOMACAO E CONTROLE                              2
ENGENHARIA MECATRÔNICA - ENGENHARIA DE CONTROLE E AU

In [14]:
ev_comp = evasao_ponderada(comps, par_datas)

ENGENHARIA DE COMPUTAÇÃO                                   862
ENGENHARIA DA COMPUTAÇÃO                                   261
ENGENHARIA DE COMPUTAÇÃO E INFORMAÇÃO                        7
ENGENHARIA DE COMPUTAÇÃO - ÊNFASE SISTEMAS CORPORATIVOS      3
ENGENHARIA DE COMPUTAÇÃO - ENGENHARIA DA COMPUTAÇÃO          3
ENGENHARIA DA COMPUTAÇÃO - COMPUTAÇÃO APLICADA               2
ENGENHARIA DA COMPUTAÇÃO - AUTOMAÇÃO INDUSTRIAL              2
ENGENHARIA DA COMPUTAÇÃO - SISTEMAS E COMPUTAÇÃO             2
ENGENHARIA DA COMPUTAÇÃO - TELEMÁTICA                        1
Name: NO_CURSO, dtype: int64


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in double_scalars


In [15]:
sum(ev_comp)

0.75360439788403677

In [10]:
ev_mec = evasao_ponderada(mec, par_datas)


ENGENHARIA MECÂNICA                                    2080
ENGENHARIA MECÂNICA AERONÁUTICA                          13
ENGENHARIA MECÂNICA - ÊNFASE EM MATERIAIS METÁLICOS       7
ENGENHARIA MECÂNICA EMPRESARIAL                           7
ENGENHARIA MECÂNICA - ÊNFASE EM MECÂNICA PLENA            7
ENGENHARIA MECÂNICA E DE AUTOMÓVEIS                       7
ENGENHARIA MECÂNICA - ÊNFASE EM PROJETOS                  6
ENGENHARIA MECÂNICA - ENGENHARIA MECÂNICA                 6
ENGENHARIA MECÂNICA E DE ARMAMENTO                        5
ENGENHARIA MECÂNICA - ÊNFASE EM AERONAVES                 4
ABI - ENGENHARIA MECÂNICA                                 3
ENGENHARIA MECÂNICA DE ARMAMENTO                          2
ENGENHARIA INDUSTRIAL MECÂNICA                            2
ENGENHARIA MECÂNICA - MECÂNICA E AUTOMOBILÍSTICA          2
ENGENHARIA MECÂNICA - CONTROLE E AUTOMAÇÃO                2
ENGENHARIA MECÂNICA - ÊNFASE EM MECATRÔNICA               2
ENGENHARIA - ENGENHARIA MECÂNICA        

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in double_scalars


In [11]:
sum(ev_mec)

0.68598530254860079

pd.crosstab(data.CO_IES, data.NO_CURSO)

In [12]:
ev_mex = evasao_ponderada(mex, par_datas)


ENGENHARIA DE CONTROLE E AUTOMAÇÃO                                         966
ENGENHARIA MECATRÔNICA                                                     187
ENGENHARIA DE CONTROLE E AUTOMACAO                                          41
ENGENHARIA DE CONTROLE E AUTOMAÇÃO - MECATRÔNICA                            38
ENGENHARIA - ENGENHARIA DE CONTROLE E AUTOMACAO                             11
ENGENHARIA DE AUTOMAÇÃO E CONTROLE                                          11
ENGENHARIA DE CONTROLE E AUTOMACAO  MECATRONICA                             10
ENGENHARIA DE CONTROLE E AUTOMACAO MECATRONICA                               8
ENGENHARIA DE MECATRÔNICA                                                    7
ENGENHARIA DE CONTROLE E AUTOMAÇÃO DE PROCESSOS                              7
MECATRÔNICA                                                                  3
ENGENHARIA - ENGENHARIA DE AUTOMACAO E CONTROLE                              2
ENGENHARIA MECATRÔNICA - ENGENHARIA DE CONTROLE E AU

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in double_scalars


In [13]:
sum(ev_mex)

0.7264583909317115